In [1]:
import pandas as pd
import os 
import joblib
import warnings;warnings.simplefilter('ignore')
from tqdm import tqdm_notebook as tqdm
root = 'PROII20200310/'
excel_list = os.listdir('PROII20200310')[2:-1]
excel_list

['Aroma-1_Unit_000_Simulation_Results_Rerun_001_050_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_051_100_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_101_150_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_151_200_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_201_250_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_251_300_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_301_350_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_351_400_Yu Final.xlsx',
 'Aroma-1_Unit_000_Simulation_Results_Rerun_401_450_Yu Final.xlsx']

In [2]:
df = pd.read_excel(root+excel_list[0],sheet_name='001')
component_list = df.iloc[22:22+54,2].values.tolist()
component_list = [i.strip() for i in component_list]

# clean_data

In [3]:
def clean_data(df):
    df = df.T.reset_index()
    idx = df.iloc[5:,1].values
    prefix = df.iloc[1,9:-6].values
    prefix = pd.Series(prefix).fillna(method='ffill').values
    middle = df.iloc[2,9:-6].values
    suffix = df.iloc[3,9:-6].values
    col = [ str(p).strip()+'_'+str(m).strip()+'_'+str(s).strip() for p,m,s in zip(prefix,middle,suffix)]
    df = df.iloc[5:,9:-6].reset_index(drop=True)
    df.columns = col
    df.index = idx
    return df

In [4]:
temp = clean_data(df)
print(temp.shape)
temp.head()

(33, 274)


,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Case Conditions_Feed Rate to C013 (Results)_m3/hr,Case Conditions_C7+ in Light End (Results)_wt%,Case Conditions_Heart Cut Prod. Rate (Results)_m3/hr,Case Conditions_C6P- in Heart Cut (Results)_wt%,Case Conditions_C10+ in Heart Cut (Results)_wt%,Naphtha Properties_Naphtha Feed Rate_m3/hr,...,Heavy End Product Properties_3-Methylnonane_wt%,Heavy End Product Properties_tert-Butylcyclohexane_wt%,Heavy End Product Properties_n-Decane_wt%,"Heavy End Product Properties_1,2-Diethylbenzene_wt%",Heavy End Product Properties_n-Undecane_wt%,Heavy End Product Properties_n-Pentylbenzene_wt%,Heavy End Product Properties_n-Dodecane_wt%,Heavy End Product Properties_n-Hexylbenzene_wt%,Heavy End Product Properties_n-Tridecane_wt%,Heavy End Product Properties_Overall_wt%
001-01,10,2.5,100,1,10,2.50006,100,1.00043,6.13864,118,...,13.9561,2.49718,5.28188,3.508,1.0905,0.220558,0.0205293,0.0413709,0,100
001-02,10,5,100,1,10,4.99988,100,1.0004,6.24905,118,...,14.1675,2.53598,5.36998,3.5699,1.11178,0.224973,0.020956,0.0422407,0,100
001-03,10,10,100,1,10,10,100,0.999993,6.37605,118,...,14.7221,2.63723,5.5944,3.72486,1.16336,0.235584,0.021969,0.044298,0,100
001-04,10,2.5,100,1.5,10,2.50002,100,1.50029,6.31633,118,...,12.4087,2.21624,4.67997,3.10153,0.96164,0.194369,0.0180776,0.0364207,0,100
001-05,10,5,100,1.5,10,5,99.9998,1.49996,6.41686,118,...,12.534,2.23931,4.73287,3.13906,0.974726,0.197096,0.0183426,0.0369624,0,100


# for loop load all data

In [5]:
train = pd.DataFrame()
error = []

# load data
for file_name in tqdm(excel_list):
    # get data
    data = pd.read_excel(root+file_name,sheet_name = None)
    # get sheet list
    start,end = int(file_name.split('_')[-3]),int(file_name.split('_')[-2])
    sheet_names = [str(i).zfill(3) for i in range(start,end+1)]
    # get sheet
    for i in tqdm(sheet_names):
        try:
            train = train.append(clean_data(data.get(i)))
        except:
            error.append((file_name,i))
    print(train.shape)


(1650, 277)



(3302, 277)



(4960, 277)



(6618, 277)



(8268, 277)



(9918, 277)



(11574, 277)



(13226, 277)



(14876, 277)



In [6]:
error

[]

# check and save

In [7]:
print(train.shape)
train.head()

(14876, 277)


,Case Conditions_C10+ in Heart Cut (Results)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%,Case Conditions_C6P- in Heart Cut (Results)_wt%,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C7+ in Light End (Results)_wt%,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Feed Rate to C013 (Results)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Results)_m3/hr,Column Temp Profile_C011 Btm Temp_oC,...,Operating Pressure_C013 Btm_kg/cm2_g,Operating Pressure_C013 Ovhd_kg/cm2_g,Operating Pressure_V012_kg/cm2_g,Specifications_Spec 1 : C7+ in Light End_wt%,Specifications_Spec 2 : Heart Cut Prod. Rate_m3/hr,Specifications_Spec 3 : C6P- in Heart Cut_wt%,Yield Summary_Heart Cut Prod. Rate_m3/hr,Yield Summary_Heavy End Prod. Rate_m3/hr,Yield Summary_Light End Prod. Rate_m3/hr,Yield Summary_Reflux Rate_m3/hr
001-01,6.13864,1,1.00043,2.5,2.50006,10,10,100,100,134.701,...,1.0767,0.844483,0.63,2.50006,100,1.00043,100,10.002,7.9979,47.0316
001-02,6.24905,1,1.0004,5,4.99988,10,10,100,100,135.046,...,1.06545,0.841137,0.63,4.99988,100,1.0004,100,9.55389,8.44599,44.0333
001-03,6.37605,1,0.999993,10,10,10,10,100,100,135.526,...,1.05466,0.83775,0.63,10,100,0.999993,100,8.85661,9.14338,41.0794
001-04,6.31633,1.5,1.50029,2.5,2.50002,10,10,100,100,133.073,...,1.04855,0.836441,0.63,2.50002,100,1.50029,100,10.7157,7.28407,41.4398
001-05,6.41686,1.5,1.49996,5,5,10,10,100,99.9998,133.386,...,1.03987,0.833844,0.63,5,99.9998,1.49996,99.9998,10.3031,7.69713,38.8748


In [8]:
xna_col = ['Naphtha Properties_{}_wt%'.format(i.strip()) for i in component_list]
xle_col = ['Light End Product Properties_{}_wt%'.format(i.strip()) for i in component_list]
xhc_col = ['Heart Cut Product Properties_{}_wt%'.format(i.strip()) for i in component_list]
xhe_col = ['Heavy End Product Properties_{}_wt%'.format(i.strip()) for i in component_list]

In [9]:
xna = train[xna_col]
xle = train[xle_col]
xhc = train[xhc_col]
xhe = train[xhe_col]

In [10]:
print(train.shape)
train = train.drop(xna+xle+xhc+xhe,axis=1)
train = pd.concat([train,xna,xle,xhc,xhe],axis=1)
print(train.shape)

(14876, 277)
(14876, 277)


In [11]:
fna_col = ['Naphtha Properties_Naphtha Feed Rate_ton/hr']
fle_col = ['Light End Product Properties_Light End Prod. Rate_ton/hr']
fhc_col = ['Heart Cut Product Properties_Heart Cut Prod. Rate_ton/hr']
fhe_col = ['Heavy End Product Properties_Heavy End Product Rate_ton/hr']

In [12]:
col_names = {}
col_names['component_list'] = component_list
#==========================
col_names['xna'] = xna_col
#=========================
col_names['xle'] = xle_col
col_names['xhc'] = xhc_col
col_names['xhe'] = xhe_col
#=========================
col_names['fna'] = fna_col
#=========================
col_names['fle'] = fle_col
col_names['fhc'] = fhc_col
col_names['fhe'] = fhe_col
#=========================
col_names['all'] = xna_col+xle_col+xhc_col+xhe_col+fna_col+fle_col+fhc_col+fhe_col

In [13]:
train.to_csv('./dataset/train.csv')
joblib.dump(col_names,'./dataset/col_names.pkl')

['./dataset/col_names.pkl']